# Purpose: 
Perform LSTM model training on office and general

In [1]:
import pandas as pd
import re 
import numpy as np 
import spacy 
# from wordcloud import WordCloud
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokens import Doc
import re
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import LSTM, Dense,Embedding, Input, Dropout,BatchNormalization, Bidirectional
from tensorflow import keras
from tensorflow.keras import backend as be
from tensorflow.keras import Sequential, Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from kerastuner import HyperModel

2023-05-27 18:21:43.180371: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# loading the pre-trained embedding file - glove.6B.100d.txt into embeddings_index matrix 
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [1]:
class MyHyperModel(HyperModel):
    def __init__(self, vocab_size, embedding_matrix, hyperparameters):
        self.vocab_size = vocab_size
        self.embedding_matrix = embedding_matrix
        self.hyperparameters = hyperparameters
    
    def build(self, hp):
        input_ = layers.Input(shape=(512,))
        embeds = layers.Embedding(self.vocab_size + 1, 100, input_length=512, weights=[self.embedding_matrix], trainable=False)(input_)
        drop = layers.Dropout(dropout_rate)(embeds)
        lstm_1 = layers.LSTM(units)(drop)
        weight_dense = layers.Dense(units=256, activation='relu')(lstm_1)
        drop_2 = layers.Dropout(units)(weight_dense)
        output_layer = layers.Dense(1, activation='sigmoid')(drop_2)
        model = keras.Model(inputs=input_, outputs=output_layer)
        
        # Set hyperparameters
        learning_rate = hp.Choice('learning_rate', values=self.hyperparameters['learning_rate'])
        units = hp.Choice('dropout_rate', values=self.hyperparameters['dropout_rate'])
        units = hp.Choice('units', values=self.hyperparameters['units'])
        lstm_units = hp.Choice('lstm_units', values=self.hyperparameters['lstm_units'])
        
        # Compile the model with the hyperparameters
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        
        return model



The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [2]:
hyperparameters = {
    'learning_rate': [0.1, 0.01, 0.001],
    'dropout_rate': [0.1,0.3, 0.4, 0.5],
    'units': [128, 256, 512],
    'lstm_units': [256, 350, 512]
}
hypermodel = MyHyperModel(vocab_size, embedding_matrix)
tuner = kt.Hyperband(hypermodel, objective='val_accuracy', max_epochs=50, factor=3, directory='my_dir', project_name='my_project')
tuner.search(x_train, y_train, validation_data=(x_val, y_val), epochs=10)
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
model = tuner.hypermodel.build(best_hyperparameters)


The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [3]:
def get_train_test_split(df, stratify_col, test_size=0.3, random_state=42): 
     """
    Split the DataFrame into training and test sets.

    Args:
        df (DataFrame): The input DataFrame to be split.
        stratify_col: The column used for stratified splitting.
        test_size (float): The proportion of the data to be included in the test set.
        random_state (int): The random seed for reproducibility.

    Returns:
        tuple: A tuple containing the training and test sets.
    """
    train, test_df = train_test_split(df, test_size= test_size, random_state=random_state, stratify= stratify_col)
    return train, test_df

def load_glove_embedding(glove_path): 
    """
    Load the GloVe word embeddings from the specified path.

    Args:
        glove_path (str): The path to the GloVe word embeddings file.

    Returns:
        dict: A dictionary mapping words to their corresponding word vectors.
    """
    f = open(glove_path, encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index


def create_lstm_train_data(lstm_train_df,lstm_test_df, text_col_name, label_col_name): 
    X_train =lstm_train_df[text_col_name]
    y_train= lstm_train_df[label_col_name]
    X_test= lstm_test_df[text_col_name]
    y_test= lstm_test_df[label_col_name]
    #set up the tokenizer
    # Encoding the reviews using Tokenizer from tensorflow preprocessing 
    #set up the tokenizer
    MAX_VOCAB_SIZE = 15000 
    # Set up tokenizer and keep max_vocab_size based on the word frequency, oov_token is given to be added to the word_index during the text_to_sequences call
    tokenizer = Tokenizer(num_words = MAX_VOCAB_SIZE,oov_token="<oov>", lower=True)
    tokenizer.fit_on_texts(X_train)
    # determination of the  training data vocabulary size
    vocab_index = tokenizer.word_index
    #print(word_index)
    V = len(vocab_index)
    ##create sequences of reviews
    # transformation based on text present in the reviews to sequence of integers
    X_seq_train = tokenizer.texts_to_sequences(X_train)
    X_seq_test =  tokenizer.texts_to_sequences(X_test)
    #create padded sequences
    # experimented with the max length and embedding size to get better performance metrics 
    maxlen= 512
    embed_size = 100
    # perform padding to the same length:
    # Taking default value of pre in padding, for padding 
    # Truncating: removal of values larger than maxlen, used post. 
    pad_train=pad_sequences(X_seq_train,truncating = 'post', padding = 'pre',maxlen=512)
    pad_test=pad_sequences(X_seq_test,truncating = 'post', padding = 'pre',maxlen=512)
    embedding_matrix = create_embedding_matrix(tokenizer.word_index, V)
    return pad_train, pad_test, y_train,y_test, V, embedding_matrix


def create_embedding_matrix( word_index, vocab_length): 
    embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words
    word_index = word_index
    embedding_matrix = np.zeros((vocab_length+1, embedding_dim)) # embedding_matrix.shape (10000, 100)
    embeddings_index = load_glove_embedding('glove.6B.100d.txt')
    #get corresponding word embeddings/weights for imdb review corpus amd create embedding_matrix
    for word, i in word_index.items():
        if i < vocab_length:
            embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Disagreement With Ratings:

In [6]:
# read the file: 
disagreement_with_ratings_df = pd.read_csv('/Users/kartikvijay/Documents/MADS/Thesis pt.2/xgboost_LSTMs/general_training_data/disagreement_with_ratings_general.csv')
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(disagreement_with_ratings_df,stratify_col=disagreement_with_ratings_df['disagreement_with_ratings_ohe'])


pad_train, pad_test, y_train,y_test, V, embedding_matrix= create_lstm_train_data(train,
                                                                                  test_df, 
                                                                                 'full_review',
                                                                                 'disagreement_with_ratings_ohe'
                                                                                 )

Found 400000 word vectors.


In [10]:
be.clear_session()
with tf.device('/GPU:0'):
     # takes input with shape as specified,   
    input_ = Input(shape=(512,))
    # creation of the embedding layer and setting pre-trained word weights and trainable
    # parameter as false
    embeds = Embedding(V+1, 100,input_length = 512, weights=[embedding_matrix], trainable = False)(input_)
    # adding dropout for regularization
    drop = tf.keras.layers.Dropout(0.3)(embeds)
    # Creation of lstm layer with 350 as dimension of the output space.
    lstm_1 = LSTM(256)(drop)
    # add fully connected dense layer to combine and weight the features that are extracted from LSTM. Addition
    # of the dense layer allows for better regularization as well by the addition of parameters. 
    # Treating the output from lstm as a regression problem
    weight_dense = Dense(units=256, activation='relu')(lstm_1)
    # added dropout for further regularization
    drop_2 = Dropout(0.1)(weight_dense)
    # output layer with sigmoid activation for binary classification
    output_layer = Dense(1, activation='sigmoid')(drop_2)
    # Finally, create a model instance!
    model = Model(inputs = input_, outputs = output_layer)
    model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 embedding (Embedding)       (None, 512, 100)          3496300   
                                                                 
 dropout (Dropout)           (None, 512, 100)          0         
                                                                 
 lstm (LSTM)                 (None, 350)               631400    
                                                                 
 dense (Dense)               (None, 256)               89856     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257   

2023-05-11 11:46:05.399852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 11:46:05.401425: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 11:46:05.402390: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [11]:
# monitoring validation loss for early stopping to avoid overfitting with patience of 4
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                mode='min', 
                               patience = 4 )
# utilization of adam optimization
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# use gpu and fit model, considered a validation_split of 10% from the training dataset
with tf.device('/GPU:0'):

    lstm_model_glove = model.fit(pad_train,y_train, 
                    validation_split= 0.1,  
                  epochs = 100, 
                  batch_size = 64, 
                  callbacks =[early_stop])

Epoch 1/100


2023-05-11 11:46:05.839815: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 11:46:05.841594: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 11:46:05.842891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

111/111 [==============================] - ETA: 0s - loss: 0.2071 - accuracy: 0.9440

2023-05-11 11:46:44.453111: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 11:46:44.454563: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 11:46:44.455530: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

111/111 [==============================] - 40s 341ms/step - loss: 0.2071 - accuracy: 0.9440 - val_loss: 0.1867 - val_accuracy: 0.9555
Epoch 2/100
111/111 [==============================] - 37s 337ms/step - loss: 0.1940 - accuracy: 0.9519 - val_loss: 0.1845 - val_accuracy: 0.9555
Epoch 3/100
111/111 [==============================] - 36s 329ms/step - loss: 0.1919 - accuracy: 0.9519 - val_loss: 0.1889 - val_accuracy: 0.9555
Epoch 4/100
111/111 [==============================] - 39s 347ms/step - loss: 0.1895 - accuracy: 0.9518 - val_loss: 0.1844 - val_accuracy: 0.9555
Epoch 5/100
111/111 [==============================] - 39s 351ms/step - loss: 0.1903 - accuracy: 0.9519 - val_loss: 0.1846 - val_accuracy: 0.9555
Epoch 6/100
111/111 [==============================] - 40s 358ms/step - loss: 0.1882 - accuracy: 0.9519 - val_loss: 0.1880 - val_accuracy: 0.9555
Epoch 7/100
111/111 [==============================] - 38s 346ms/step - loss: 0.1809 - accuracy: 0.9519 - val_loss: 0.1783 - val_accurac

In [12]:
# make predictions on the test data: 
y_pred = model.predict(pad_test)
y_pred = np.round(y_pred)
print(classification_report(y_pred, y_test))

2023-05-11 11:53:11.020024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 11:53:11.021718: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 11:53:11.023010: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

106/106 [==============================] - 7s 59ms/step
              precision    recall  f1-score   support

         0.0       1.00      0.95      0.98      3369
         1.0       0.00      0.00      0.00         0

    accuracy                           0.95      3369
   macro avg       0.50      0.48      0.49      3369
weighted avg       1.00      0.95      0.98      3369



/Users/kartikvijay/anaconda3/envs/kvj_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kartikvijay/anaconda3/envs/kvj_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kartikvijay/anaconda3/envs/kvj_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

# Rating Management Explicit

In [19]:
# read the file: 
rating_management_explicit_df = pd.read_csv('/Users/kartikvijay/Documents/MADS/Thesis pt.2/xgboost_LSTMs/general_training_data/rating_managment_explicit_general.csv')
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(rating_management_explicit_df,stratify_col=rating_management_explicit_df['rating_managment_explicit_ohe'])


pad_train, pad_test, y_train,y_test, V, embedding_matrix= create_lstm_train_data(train,
                                                                                  test_df, 
                                                                                 'full_review',
                                                                                 'rating_managment_explicit_ohe'
                                                                                 )


Found 400000 word vectors.


In [20]:
be.clear_session()
with tf.device('/GPU:0'):
     # takes input with shape as specified,   
    input_ = Input(shape=(512,))
    # creation of the embedding layer and setting pre-trained word weights and trainable
    # parameter as false
    embeds = Embedding(V+1, 100,input_length = 512, weights=[embedding_matrix], trainable = False)(input_)
    # adding dropout for regularization
    drop = tf.keras.layers.Dropout(0.02)(embeds)
    # Creation of lstm layer with 350 as dimension of the output space.
    lstm_1 = LSTM(512)(drop)
    # add fully connected dense layer to combine and weight the features that are extracted from LSTM. Addition
    # of the dense layer allows for better regularization as well by the addition of parameters. 
    # Treating the output from lstm as a regression problem
    weight_dense = Dense(units=256, activation='relu')(lstm_1)
    # added dropout for further regularization
    drop_2 = Dropout(0.2)(weight_dense)
    # output layer with sigmoid activation for binary classification
    output_layer = Dense(1, activation='sigmoid')(drop_2)
    # Finally, create a model instance!
    model = Model(inputs = input_, outputs = output_layer)
    model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 embedding (Embedding)       (None, 512, 100)          3466900   
                                                                 
 dropout (Dropout)           (None, 512, 100)          0         
                                                                 
 lstm (LSTM)                 (None, 350)               631400    
                                                                 
 dense (Dense)               (None, 256)               89856     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257   

2023-05-11 11:56:54.465835: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 11:56:54.467369: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 11:56:54.468417: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [21]:
# monitoring validation loss for early stopping to avoid overfitting with patience of 4
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                mode='min', 
                               patience = 4 )
# utilization of adam optimization
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# use gpu and fit model, considered a validation_split of 10% from the training dataset
with tf.device('/GPU:0'):

    lstm_model_glove = model.fit(pad_train,y_train, 
                    validation_split= 0.1,  
                  epochs = 100, 
                  batch_size = 64, 
                  callbacks =[early_stop])

Epoch 1/100


2023-05-11 11:57:02.785181: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 11:57:02.786967: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 11:57:02.788077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

111/111 [==============================] - ETA: 0s - loss: 0.0934 - accuracy: 0.9802

2023-05-11 11:57:40.552923: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 11:57:40.554399: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 11:57:40.555429: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

111/111 [==============================] - 39s 333ms/step - loss: 0.0934 - accuracy: 0.9802 - val_loss: 0.0851 - val_accuracy: 0.9835
Epoch 2/100
111/111 [==============================] - 36s 319ms/step - loss: 0.0809 - accuracy: 0.9847 - val_loss: 0.0848 - val_accuracy: 0.9835
Epoch 3/100
111/111 [==============================] - 37s 329ms/step - loss: 0.0774 - accuracy: 0.9847 - val_loss: 0.0878 - val_accuracy: 0.9835
Epoch 4/100
111/111 [==============================] - 36s 326ms/step - loss: 0.0795 - accuracy: 0.9847 - val_loss: 0.0795 - val_accuracy: 0.9835
Epoch 5/100
111/111 [==============================] - 36s 323ms/step - loss: 0.0746 - accuracy: 0.9847 - val_loss: 0.0779 - val_accuracy: 0.9835
Epoch 6/100
111/111 [==============================] - 36s 327ms/step - loss: 0.0693 - accuracy: 0.9847 - val_loss: 0.0859 - val_accuracy: 0.9835
Epoch 7/100
111/111 [==============================] - 38s 341ms/step - loss: 0.0728 - accuracy: 0.9847 - val_loss: 0.0673 - val_accurac

In [22]:
# make predictions on the test data: 
y_pred = model.predict(pad_test)
y_pred = np.round(y_pred)
print(classification_report(y_pred, y_test))

2023-05-11 12:08:01.902706: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 12:08:01.904466: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 12:08:01.905681: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

106/106 [==============================] - 7s 57ms/step
              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99      3360
         1.0       0.13      0.78      0.23         9

    accuracy                           0.99      3369
   macro avg       0.57      0.88      0.61      3369
weighted avg       1.00      0.99      0.99      3369



# Wish I read Reviews:


In [23]:
read_reviews_df = pd.read_csv('/Users/kartikvijay/Documents/MADS/Thesis pt.2/xgboost_LSTMs/general_training_data/read_reviews_general.csv')

# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(read_reviews_df,stratify_col=read_reviews_df['wish_I_read_reviews _ohe'])


pad_train, pad_test, y_train,y_test, V, embedding_matrix= create_lstm_train_data(train,
                                                                                test_df, 
                                                                                'full_review',
                                                                                'wish_I_read_reviews _ohe'
                                                                                 )


Found 400000 word vectors.


In [24]:
be.clear_session()
with tf.device('/GPU:0'):
     # takes input with shape as specified,   
    input_ = Input(shape=(512,))
    # creation of the embedding layer and setting pre-trained word weights and trainable
    # parameter as false
    embeds = Embedding(V+1, 100,input_length = 512, weights=[embedding_matrix], trainable = False)(input_)
    # adding dropout for regularization
    drop = tf.keras.layers.Dropout(0.1)(embeds)
    # Creation of lstm layer with 350 as dimension of the output space.
    lstm_1 = LSTM(256)(drop)
    # add fully connected dense layer to combine and weight the features that are extracted from LSTM. Addition
    # of the dense layer allows for better regularization as well by the addition of parameters. 
    # Treating the output from lstm as a regression problem
    weight_dense = Dense(units=256, activation='relu')(lstm_1)
    # added dropout for further regularization
    drop_2 = Dropout(0.1)(weight_dense)
    # output layer with sigmoid activation for binary classification
    output_layer = Dense(1, activation='sigmoid')(drop_2)
    # Finally, create a model instance!
    model = Model(inputs = input_, outputs = output_layer)
    model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 embedding (Embedding)       (None, 512, 100)          3354600   
                                                                 
 dropout (Dropout)           (None, 512, 100)          0         
                                                                 
 lstm (LSTM)                 (None, 350)               631400    
                                                                 
 dense (Dense)               (None, 256)               89856     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257   

2023-05-11 12:11:00.594227: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 12:11:00.595688: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 12:11:00.596691: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [25]:
# monitoring validation loss for early stopping to avoid overfitting with patience of 4
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                mode='min', 
                               patience = 4 )
# utilization of adam optimization
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# use gpu and fit model, considered a validation_split of 10% from the training dataset
with tf.device('/GPU:0'):

    lstm_model_glove = model.fit(pad_train,y_train, 
                    validation_split= 0.1,  
                  epochs = 100, 
                  batch_size = 64, 
                  callbacks =[early_stop])

Epoch 1/100


2023-05-11 12:11:36.819914: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 12:11:36.821715: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 12:11:36.822860: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

98/98 [==============================] - ETA: 0s - loss: 0.2212 - accuracy: 0.9428

2023-05-11 12:12:14.433105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 12:12:14.434405: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 12:12:14.435352: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

98/98 [==============================] - 39s 368ms/step - loss: 0.2212 - accuracy: 0.9428 - val_loss: 0.1895 - val_accuracy: 0.9537
Epoch 2/100
98/98 [==============================] - 31s 313ms/step - loss: 0.2110 - accuracy: 0.9470 - val_loss: 0.1792 - val_accuracy: 0.9537
Epoch 3/100
98/98 [==============================] - 31s 315ms/step - loss: 0.2057 - accuracy: 0.9470 - val_loss: 0.1923 - val_accuracy: 0.9537
Epoch 4/100
98/98 [==============================] - 31s 321ms/step - loss: 0.1949 - accuracy: 0.9469 - val_loss: 0.1707 - val_accuracy: 0.9537
Epoch 5/100
98/98 [==============================] - 35s 361ms/step - loss: 0.1806 - accuracy: 0.9469 - val_loss: 0.1795 - val_accuracy: 0.9537
Epoch 6/100
98/98 [==============================] - 32s 324ms/step - loss: 0.1700 - accuracy: 0.9470 - val_loss: 0.1557 - val_accuracy: 0.9551
Epoch 7/100
98/98 [==============================] - 37s 374ms/step - loss: 0.1461 - accuracy: 0.9483 - val_loss: 0.1327 - val_accuracy: 0.9537
Epoc

In [26]:
# make predictions on the test data: 
y_pred = model.predict(pad_test)
y_pred = np.round(y_pred)
print(classification_report(y_pred, y_test))

2023-05-11 12:21:14.322845: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 12:21:14.323962: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 12:21:14.325164: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

93/93 [==============================] - 6s 57ms/step
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98      2853
         1.0       0.55      0.82      0.66       105

    accuracy                           0.97      2958
   macro avg       0.77      0.90      0.82      2958
weighted avg       0.98      0.97      0.97      2958



# Wrong Buying:

In [4]:
wrong_buying_df = pd.read_csv('/Users/kartikvijay/Documents/MADS/Thesis pt.2/xgboost_LSTMs/general_training_data/wrong_buying_general.csv')

# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(wrong_buying_df,stratify_col=wrong_buying_df['wrong_buying_ohe'])


pad_train, pad_test, y_train,y_test, V, embedding_matrix= create_lstm_train_data(train,
                                                                                test_df, 
                                                                                'full_review',
                                                                                'wrong_buying_ohe'
                                                                                 )


Found 400000 word vectors.


In [5]:
be.clear_session()
with tf.device('/GPU:0'):
     # takes input with shape as specified,   
    input_ = Input(shape=(512,))
    # creation of the embedding layer and setting pre-trained word weights and trainable
    # parameter as false
    embeds = Embedding(V+1, 100,input_length = 512, weights=[embedding_matrix], trainable = False)(input_)
    # adding dropout for regularization
#     drop = tf.keras.layers.Dropout(0.3)(embeds)
    # Creation of lstm layer with 350 as dimension of the output space.
    lstm_1 = LSTM(350)(embeds)
    # add fully connected dense layer to combine and weight the features that are extracted from LSTM. Addition
    # of the dense layer allows for better regularization as well by the addition of parameters. 
    # Treating the output from lstm as a regression problem
    weight_dense = Dense(units=256, activation='relu')(lstm_1)
    # added dropout for further regularization
    drop_2 = Dropout(0.7)(weight_dense)
    # output layer with sigmoid activation for binary classification
    output_layer = Dense(1, activation='sigmoid')(drop_2)
    # Finally, create a model instance!
    model = Model(inputs = input_, outputs = output_layer)
    model.summary()

Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-05-27 18:22:26.646390: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-27 18:22:26.647265: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 embedding (Embedding)       (None, 512, 100)          3528400   
                                                                 
 lstm (LSTM)                 (None, 350)               631400    
                                                                 
 dense (Dense)               (None, 256)               89856     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 4,249,913
Trainable params: 721,513
Non-trainab

2023-05-27 18:22:27.139348: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-27 18:22:27.140225: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-27 18:22:27.140892: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [6]:
# monitoring validation loss for early stopping to avoid overfitting with patience of 4
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                mode='min', 
                               patience = 3 )
# utilization of adam optimization
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# use gpu and fit model, considered a validation_split of 10% from the training dataset
with tf.device('/GPU:0'):

    lstm_model_glove = model.fit(pad_train,y_train, 
                    validation_split= 0.1,  
                  epochs = 100, 
                  batch_size = 64, 
                  callbacks =[early_stop])

Epoch 1/100


2023-05-27 18:22:41.171277: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-27 18:22:41.172241: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-27 18:22:41.172881: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

111/111 [==============================] - ETA: 0s - loss: 0.3180 - accuracy: 0.9033

2023-05-27 18:23:02.132119: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-27 18:23:02.133201: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-27 18:23:02.133865: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

111/111 [==============================] - 22s 181ms/step - loss: 0.3180 - accuracy: 0.9033 - val_loss: 0.2520 - val_accuracy: 0.9301
Epoch 2/100
111/111 [==============================] - 19s 173ms/step - loss: 0.3040 - accuracy: 0.9074 - val_loss: 0.2490 - val_accuracy: 0.9301
Epoch 3/100
111/111 [==============================] - 19s 172ms/step - loss: 0.3030 - accuracy: 0.9074 - val_loss: 0.2499 - val_accuracy: 0.9301
Epoch 4/100
111/111 [==============================] - 19s 173ms/step - loss: 0.2926 - accuracy: 0.9075 - val_loss: 0.2494 - val_accuracy: 0.9301
Epoch 5/100
111/111 [==============================] - 19s 174ms/step - loss: 0.2761 - accuracy: 0.9075 - val_loss: 0.2417 - val_accuracy: 0.9288
Epoch 6/100
111/111 [==============================] - 19s 169ms/step - loss: 0.2508 - accuracy: 0.9075 - val_loss: 0.2111 - val_accuracy: 0.9288
Epoch 7/100
111/111 [==============================] - 19s 169ms/step - loss: 0.2207 - accuracy: 0.9084 - val_loss: 0.2043 - val_accurac

In [7]:
# make predictions on the test data: 
y_pred = model.predict(pad_test)
y_pred = np.round(y_pred)
print(classification_report(y_pred, y_test))

  1/106 [..............................] - ETA: 35s

2023-05-27 18:26:51.415686: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-27 18:26:51.416743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-27 18:26:51.417559: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

106/106 [==============================] - 4s 33ms/step
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95      2954
         1.0       0.65      0.48      0.55       415

    accuracy                           0.90      3369
   macro avg       0.79      0.72      0.75      3369
weighted avg       0.90      0.90      0.90      3369



# Zero Stars:

In [32]:
zero_stars_df = pd.read_csv('/Users/kartikvijay/Documents/MADS/Thesis pt.2/xgboost_LSTMs/general_training_data/zero_star_general.csv')
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(zero_stars_df,stratify_col=zero_stars_df['zero_star_ohe'])


pad_train, pad_test, y_train,y_test, V, embedding_matrix= create_lstm_train_data(train,
                                                                                test_df, 
                                                                                'full_review',
                                                                                'zero_star_ohe'
                                                                                 )


Found 400000 word vectors.


In [33]:
be.clear_session()
with tf.device('/GPU:0'):
     # takes input with shape as specified,   
    input_ = Input(shape=(512,))
    # creation of the embedding layer and setting pre-trained word weights and trainable
    # parameter as false
    embeds = Embedding(V+1, 100,input_length = 512, weights=[embedding_matrix], trainable = False)(input_)
    # adding dropout for regularization
    drop = tf.keras.layers.Dropout(0.3)(embeds)
    # Creation of lstm layer with 350 as dimension of the output space.
    lstm_1 = LSTM(350)(drop)
    # add fully connected dense layer to combine and weight the features that are extracted from LSTM. Addition
    # of the dense layer allows for better regularization as well by the addition of parameters. 
    # Treating the output from lstm as a regression problem
    weight_dense = Dense(units=256, activation='relu')(lstm_1)
    # added dropout for further regularization
    drop_2 = Dropout(0.5)(weight_dense)
    # output layer with sigmoid activation for binary classification
    output_layer = Dense(1, activation='sigmoid')(drop_2)
    # Finally, create a model instance!
    model = Model(inputs = input_, outputs = output_layer)
    model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 embedding (Embedding)       (None, 512, 100)          3512900   
                                                                 
 dropout (Dropout)           (None, 512, 100)          0         
                                                                 
 lstm (LSTM)                 (None, 350)               631400    
                                                                 
 dense (Dense)               (None, 256)               89856     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257   

2023-05-11 12:32:03.953597: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 12:32:03.954979: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 12:32:03.956371: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [34]:
# monitoring validation loss for early stopping to avoid overfitting with patience of 4
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                mode='min', 
                               patience = 4 )
# utilization of adam optimization
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# use gpu and fit model, considered a validation_split of 10% from the training dataset
with tf.device('/GPU:0'):

    lstm_model_glove = model.fit(pad_train,y_train, 
                    validation_split= 0.1,  
                  epochs = 100, 
                  batch_size = 64, 
                  callbacks =[early_stop])

Epoch 1/100


2023-05-11 12:32:17.871994: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 12:32:17.873679: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 12:32:17.874821: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

111/111 [==============================] - ETA: 0s - loss: 0.3566 - accuracy: 0.8763

2023-05-11 12:32:57.318490: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 12:32:57.319556: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 12:32:57.320590: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

111/111 [==============================] - 41s 349ms/step - loss: 0.3566 - accuracy: 0.8763 - val_loss: 0.2462 - val_accuracy: 0.9136
Epoch 2/100
111/111 [==============================] - 38s 341ms/step - loss: 0.2488 - accuracy: 0.8978 - val_loss: 0.1784 - val_accuracy: 0.8945
Epoch 3/100
111/111 [==============================] - 39s 347ms/step - loss: 0.3187 - accuracy: 0.8848 - val_loss: 0.3165 - val_accuracy: 0.9009
Epoch 4/100
111/111 [==============================] - 39s 346ms/step - loss: 0.2930 - accuracy: 0.9056 - val_loss: 0.2718 - val_accuracy: 0.9136
Epoch 5/100
111/111 [==============================] - 37s 334ms/step - loss: 0.2385 - accuracy: 0.9208 - val_loss: 0.1493 - val_accuracy: 0.9454
Epoch 6/100
111/111 [==============================] - 40s 364ms/step - loss: 0.1605 - accuracy: 0.9450 - val_loss: 0.1198 - val_accuracy: 0.9657
Epoch 7/100
111/111 [==============================] - 39s 352ms/step - loss: 0.1593 - accuracy: 0.9490 - val_loss: 0.1191 - val_accurac

In [35]:
# make predictions on the test data: 
y_pred = model.predict(pad_test)
y_pred = np.round(y_pred)
print(classification_report(y_pred, y_test))

2023-05-11 12:43:17.708404: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-11 12:43:17.709762: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-11 12:43:17.710663: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

106/106 [==============================] - 8s 68ms/step
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98      2995
         1.0       0.84      0.93      0.88       374

    accuracy                           0.97      3369
   macro avg       0.92      0.96      0.93      3369
weighted avg       0.97      0.97      0.97      3369



In [36]:
print('End of Notebook!')

End of Notebook!
